In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#NOTE: IS_SUCCESSFUL should be my target

#FEATURES will be: AFFILIATION, ORGANIZATION, USE_CASE, STATUS, INCOME_AMT, ASK_AMT,

#Dropped will be: EIN, NAME, APPLICATION_TYPE, SPECIAL, CONSIDERATIONS, CLASSIFICATION

charity_relevent_df = charity_df.drop(columns=["EIN","NAME","APPLICATION_TYPE","SPECIAL_CONSIDERATIONS","CLASSIFICATION"])
charity_relevent_df.head()

,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Independent,ProductDev,Association,1,0,5000,1
1,Independent,Preservation,Co-operative,1,1-9999,108590,1
2,CompanySponsored,ProductDev,Association,1,0,5000,0
3,CompanySponsored,Preservation,Trust,1,10000-24999,6692,1
4,Independent,Heathcare,Trust,1,100000-499999,142590,1


In [3]:
#Lets make sure that none of the categorical data needs bucketing


# Generate our categorical variable list
charity_cat = charity_relevent_df.dtypes[charity_relevent_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_relevent_df[charity_cat].nunique()

AFFILIATION     6
USE_CASE        5
ORGANIZATION    4
INCOME_AMT      9
dtype: int64

In [4]:
# Check the unique value counts to see if bucketing required
charity_relevent_df.USE_CASE.value_counts()


Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [5]:
charity_relevent_df.ORGANIZATION.value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [6]:
# I AM GOING TO DECIDE NOT TO BUCKET FOR SIMPLICITY

In [7]:
#Lets encode data now:

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_relevent_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_relevent_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
charity_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'AFFILIATION_CompanySponsored',
       'AFFILIATION_Family/Parent', 'AFFILIATION_Independent',
       'AFFILIATION_National', 'AFFILIATION_Other', 'AFFILIATION_Regional',
       'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other',
       'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M'],
      dtype='object')

In [15]:
# Remove the target features from the data
y = charity_relevent_df['IS_SUCCESSFUL'].values
X = charity_df.drop(columns=["IS_SUCCESSFUL"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# WE WILL TRY FIRST TO USE A DEEP LEARNING MODEL.
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [17]:
#train and evaluate the DLL

# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 2s 91us/sample - loss: 0.6371 - accuracy: 0.6704
Epoch 2/50
25724/25724 [==============================] - 1s 48us/sample - loss: 0.6072 - accuracy: 0.6960
Epoch 3/50
25724/25724 [==============================] - 1s 46us/sample - loss: 0.6020 - accuracy: 0.6979
Epoch 4/50
25724/25724 [==============================] - 1s 47us/sample - loss: 0.6001 - accuracy: 0.6995
Epoch 5/50
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5991 - accuracy: 0.7000
Epoch 6/50
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5985 - accuracy: 0.6997
Epoch 7/50
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5983 - accuracy: 0.6994
Epoch 8/50
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5981 - accuracy: 0.7006
Epoch 9/50
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5982 - accuracy: 0.7003
Epoch 10/50
25

In [18]:
#Okay so this model topped out at about 0.7000 at one point. There is still alot of loss.

In [20]:
#We will try again, but try a different activation function: THIS time we'll try a sigmoid activation function in the HIDDEN LAYERS

# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 2s 87us/sample - loss: 0.6543 - accuracy: 0.6189
Epoch 2/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.6118 - accuracy: 0.6913
Epoch 3/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.6084 - accuracy: 0.6967
Epoch 4/50
25724/25724 [==============================] - 1s 51us/sample - loss: 0.6068 - accuracy: 0.6986
Epoch 5/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.6049 - accuracy: 0.6988
Epoch 6/50
25724/25724 [==============================] - 1s 54us/sample - loss: 0.6030 - accuracy: 0.6999
Epoch 7/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.6013 - accuracy: 0.6991
Epoch 8/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.6002 - accuracy: 0.6992
Epoch 9/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5995 - accuracy: 0.6988
Epoch 10/50
25

In [22]:
#Okay, this sucked too! Lets try going back to a "Relu" activation function in the Hidden Layers, but we'll attempt to add more layers

In [23]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 3s 105us/sample - loss: 0.6266 - accuracy: 0.6779
Epoch 2/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.6022 - accuracy: 0.6977
Epoch 3/50
25724/25724 [==============================] - 1s 55us/sample - loss: 0.6000 - accuracy: 0.6992
Epoch 4/50
25724/25724 [==============================] - 2s 58us/sample - loss: 0.5995 - accuracy: 0.6995
Epoch 5/50
25724/25724 [==============================] - 2s 58us/sample - loss: 0.5988 - accuracy: 0.6991
Epoch 6/50
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5987 - accuracy: 0.6990
Epoch 7/50
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5983 - accuracy: 0.7002
Epoch 8/50
25724/25724 [==============================] - 2s 64us/sample - loss: 0.5982 - accuracy: 0.7002
Epoch 9/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5983 - accuracy: 0.6997
Epoch 10/50
2

In [25]:
#Still having a ton of loss. The fitting is still not great in that it's still at 0.65

#Lets try increasing the epochs next

In [26]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=300) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

00
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5950 - accuracy: 0.7016
Epoch 121/300
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5950 - accuracy: 0.7023
Epoch 122/300
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5951 - accuracy: 0.7020
Epoch 123/300
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5949 - accuracy: 0.7024
Epoch 124/300
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5950 - accuracy: 0.7025
Epoch 125/300
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5951 - accuracy: 0.7019
Epoch 126/300
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5950 - accuracy: 0.7027
Epoch 127/300
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5952 - accuracy: 0.7022
Epoch 128/300
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5951 - accuracy: 0.7022
Epoch 129/300
25724/2